In [1]:
#import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import pandas as pd
import numpy as np
import json
import re
pd.options.mode.chained_assignment = None  # default='warn'

## 資料清洗

In [2]:
# read csv
df = pd.read_csv("../data/產量_臺灣地區果品生產概況(2011~2019).csv", encoding="utf8")
df

,年度,地區別,果品類別,種植株數,收穫株數,種植面積_公頃,收穫面積_公頃,每株平均產量_公斤,每公頃平均產量_公斤,產量_公噸
0,2019,新北市,香蕉,108619,102262,74.99,71.01,13.74,19793.48,1405.535
1,2019,台北市,香蕉,12830,12830,10.36,10.36,12.55,15543.15,161.027
2,2019,桃園市,香蕉,44420,43940,57.11,56.47,18.85,14666.65,828.226
3,2019,台中市,香蕉,574129,560455,629.71,614.39,15.54,14178.52,8711.141
4,2019,台南市,香蕉,1481610,1476420,1062.65,1057.46,16.26,22703.58,24008.13
...,...,...,...,...,...,...,...,...,...,...
7066,2011,基隆市,其他水果,327,327,1.01,1.01,10,3237.62,3.27
7067,2011,新竹市,其他水果,2538,2538,1.84,1.84,10.56,14560.87,26.792
7068,2011,嘉義市,其他水果,25711,25711,38.43,38.43,16.66,11147.46,428.397
7069,2011,金門縣,其他水果,0,0,0,0,0,0,0


In [3]:
df_guava = df.loc[df["果品類別"] == "番石榴"]
df_guava

,年度,地區別,果品類別,種植株數,收穫株數,種植面積_公頃,收穫面積_公頃,每株平均產量_公斤,每公頃平均產量_公斤,產量_公噸
384,2019,新北市,番石榴,38469,37544,55.15,53.61,15.89,11131.37,596.753
385,2019,台北市,番石榴,2106,2106,3.35,3.35,14.37,9031.34,30.255
386,2019,桃園市,番石榴,17584,13838,29.05,22.84,15.78,9562,218.396
387,2019,台中市,番石榴,89075,87291,129.6,127.17,26.48,18175.94,2311.434
388,2019,台南市,番石榴,1198472,1196656,1499.21,1496.94,28.27,22596.2,33825.158
...,...,...,...,...,...,...,...,...,...,...
6670,2011,基隆市,番石榴,665,665,0.96,0.96,10.21,7072.92,6.79
6671,2011,新竹市,番石榴,1174,1174,1.68,1.68,30.24,21130.95,35.5
6672,2011,嘉義市,番石榴,11178,11178,13.5,13.5,34.18,28303.56,382.098
6673,2011,金門縣,番石榴,0,0,0,0,0,0,0


### 去除不必要地區別["臺灣省","福建省"]

In [4]:
df_guava = df_guava.loc[~(df_guava["地區別"].isin(["臺灣省","福建省"]))]
df_guava

,年度,地區別,果品類別,種植株數,收穫株數,種植面積_公頃,收穫面積_公頃,每株平均產量_公斤,每公頃平均產量_公斤,產量_公噸
384,2019,新北市,番石榴,38469,37544,55.15,53.61,15.89,11131.37,596.753
385,2019,台北市,番石榴,2106,2106,3.35,3.35,14.37,9031.34,30.255
386,2019,桃園市,番石榴,17584,13838,29.05,22.84,15.78,9562,218.396
387,2019,台中市,番石榴,89075,87291,129.6,127.17,26.48,18175.94,2311.434
388,2019,台南市,番石榴,1198472,1196656,1499.21,1496.94,28.27,22596.2,33825.158
...,...,...,...,...,...,...,...,...,...,...
6670,2011,基隆市,番石榴,665,665,0.96,0.96,10.21,7072.92,6.79
6671,2011,新竹市,番石榴,1174,1174,1.68,1.68,30.24,21130.95,35.5
6672,2011,嘉義市,番石榴,11178,11178,13.5,13.5,34.18,28303.56,382.098
6673,2011,金門縣,番石榴,0,0,0,0,0,0,0


In [5]:
# 將 "-" 取代為 0
df_guava = df_guava.replace("-", 0)

In [6]:
target_cols= ['種植株數', '收穫株數', '種植面積_公頃', '收穫面積_公頃', '每株平均產量_公斤', '每公頃平均產量_公斤', '產量_公噸']
df_guava[target_cols] = df_guava[target_cols].apply(pd.to_numeric)
df_guava["產量_公噸"] = df_guava["產量_公噸"].map(lambda x: round(x ,2))
df_guava

,年度,地區別,果品類別,種植株數,收穫株數,種植面積_公頃,收穫面積_公頃,每株平均產量_公斤,每公頃平均產量_公斤,產量_公噸
384,2019,新北市,番石榴,38469,37544,55.15,53.61,15.89,11131.37,596.75
385,2019,台北市,番石榴,2106,2106,3.35,3.35,14.37,9031.34,30.25
386,2019,桃園市,番石榴,17584,13838,29.05,22.84,15.78,9562.00,218.40
387,2019,台中市,番石榴,89075,87291,129.60,127.17,26.48,18175.94,2311.43
388,2019,台南市,番石榴,1198472,1196656,1499.21,1496.94,28.27,22596.20,33825.16
...,...,...,...,...,...,...,...,...,...,...
6670,2011,基隆市,番石榴,665,665,0.96,0.96,10.21,7072.92,6.79
6671,2011,新竹市,番石榴,1174,1174,1.68,1.68,30.24,21130.95,35.50
6672,2011,嘉義市,番石榴,11178,11178,13.50,13.50,34.18,28303.56,382.10
6673,2011,金門縣,番石榴,0,0,0.00,0.00,0.00,0.00,0.00


In [7]:
df_guava.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198 entries, 384 to 6674
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   年度          198 non-null    int64  
 1   地區別         198 non-null    object 
 2   果品類別        198 non-null    object 
 3   種植株數        198 non-null    int64  
 4   收穫株數        198 non-null    int64  
 5   種植面積_公頃     198 non-null    float64
 6   收穫面積_公頃     198 non-null    float64
 7   每株平均產量_公斤   198 non-null    float64
 8   每公頃平均產量_公斤  198 non-null    float64
 9   產量_公噸       198 non-null    float64
dtypes: float64(5), int64(3), object(2)
memory usage: 17.0+ KB


In [ ]:
# 將整理後的檔案存入 cleaned data
# df_guava.to_csv("cleaned data/產量_臺灣芭樂生產概況(2011~2019).csv", index=False)